# VAE

In [5]:
import sys
import os

sys.path.append( '..' )
from VAE import *
from VAE_smac import *
from helpers.pc_stats import *

## Parameter definition

In [6]:
data_dir  = os.path.normpath(os.path.join(os.getcwd(), "../../runs/FIA/Com8_grown_together/oms"))
run_dir = os.path.normpath(os.path.join(os.getcwd(), "../../runs/VAE/hyperparameter_optimization"))
results_dir = os.path.normpath(os.path.join(os.getcwd(), "../../runs/VAE/results"))
test_configuration = False
overwrite = False
verbosity = 1
framework = "pytorch_jupyter"
outdir = Path(os.path.normpath(os.path.join(run_dir, f"smac_vae_{framework}")))

# Logging (time and steps)
last_timestamp = time.time()
step = 0
runtimes = {}


time_step(message="Setup loaded", verbosity=verbosity)

Setup loaded (0.009823322296142578s)


In [7]:
X = read_data(data_dir, verbosity=verbosity)

configuration_space = ConfigurationSpace(name="LD", seed=42)
hyperparameters = [
    Constant(       "original_dim",             X.shape[1]),
    Float(          "input_dropout",            (0.0, 0.5), default=0.25),
    Integer(        "intermediate_layers",      (1, 5), default=2),
    Integer(        "intermediate_dimension",   (10, 20), log=True, default=20),
    Categorical(    "intermediate_activation",  ["relu", "selu", "tanh", "leakyrelu"], default="selu"),
    Integer(        "latent_dimension",         (10, 11), log=False, default=10),
    Categorical(    "solver",                   ["nadam"], default="nadam"),
    Float(          "learning_rate",            (1e-4, 1e-2), log=True, default=1e-3)
]
configuration_space.add_hyperparameters(hyperparameters)
forbidden_clauses = [
    ForbiddenGreaterThanRelation(configuration_space["latent_dimension"], configuration_space["intermediate_dimension"])
]
configuration_space.add_forbidden_clauses(forbidden_clauses)
if verbosity > 0: 
    print(f"Configuration space defined with estimated {configuration_space.estimate_size()} possible combinations.\n")

Data loaded (2.3199915885925293s)
Configuration space defined with estimated inf possible combinations.



## Optimization definition

In [12]:
class Sampling(layers.Layer):
        """
        Uses (z_mean, z_log_var) to sample z, the vector encoding a digit.
        """
        def call(self, inputs):
            z_mean, z_log_var = inputs
            z_mean_shape = backend.shape(z_mean)
            batch   = z_mean_shape[0]
            dim     = z_mean_shape[1]
            epsilon = backend.random_normal(shape=(batch, dim))
            return z_mean + backend.exp(0.5 * z_log_var) * epsilon

class FIA_VAE():
    def __init__(self, config:Configuration):
        im_layers       = config["intermediate_layers"]
        im_dim          = config["intermediate_dimension"]
        activation_fun  = get_activation_function( config["intermediate_activation"] )
        
        # Encoder
        self.input      = Input(shape=(config["original_dim"],), name='encoder_input')
        self.im_enc     = Dropout( config["input_dropout"] ) (self.input)
        self.im_enc     = Dense( im_dim , activation=activation_fun ) (self.im_enc)
        for i in range(1, im_layers):                                                              # Successive halfing of layers
            if im_dim // 2**i <= config["latent_dimension"]:
                im_layers = i 
                break
            self.im_enc = Dense( im_dim // 2**i, activation=activation_fun ) (self.im_enc)
        self.mu         = Dense( config["latent_dimension"], name='latent_mu' ) (self.im_enc)
        self.sigma      = Dense( config["latent_dimension"], name='latent_sigma' ) (self.im_enc)
        self.z          = Sampling(name="Reparametrization") ( [self.mu, self.sigma] )                                 

        self.encoder = Model( self.input, [self.mu, self.sigma, self.z], name='encoder' )            # Instantiate encoder

        # Decoder
        self.decoder_input  = Input(shape=(config["latent_dimension"], ), name='decoder_input')
        prev_layer = self.decoder_input
        for i in reversed(range(1, im_layers)):
            self.im_dec =  Dense( im_dim // 2**i, activation=activation_fun) (prev_layer)
            prev_layer = self.im_dec
        self.im_dec = Dense(im_dim, activation=activation_fun) (prev_layer)
        self.output  = Dense(config["original_dim"]) (self.im_dec)

        self.decoder = Model(self.decoder_input, self.output, name='decoder')                        # Instantiate decoder

        # VAE
        self.vae_outputs = self.decoder(self.encoder(self.input)[2])
        self.vae         = Model(self.input, self.vae_outputs, name='vae')

        # Loss trackers
        self.reconstruction_loss = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss = keras.metrics.Mean(name="kl_loss")
        self.loss = keras.metrics.Mean(name="total_loss")

        # Define optimizer
        self.optimizer = get_solver( config["solver"] )( config["learning_rate"] )

        # Compile VAE
        self.vae.compile(optimizer=self.optimizer, loss=self.kl_reconstruction_loss, metrics = [ "mse" ])

    def encode(self, data):
        return self.encoder.predict(data)[2]
    
    def encode_mu(self, data):
        return self.encoder.predict(data)[0]
    
    def decode(self, data):
        return self.decoder.predict(data)
    
    def reconstruct(self, data):
        return self.decode(self.encode(data))
    
    def save_model(self, save_folder, suffix:str=""):
        self.vae.save(os.path.join(save_folder, f'VAE{suffix}.h5'))
        self.encoder.save(os.path.join(save_folder, f'VAE_encoder{suffix}.h5'))
        self.decoder.save(os.path.join(save_folder, f'VAE_decoder{suffix}.h5'))
        
    def load_vae(self, save_path):                       
        self.vae = keras.models.load_model(save_path)
        self.vae.compile(optimizer=self.optimizer, 
                         loss=self.kl_reconstruction_loss, 
                         metrics = ['mse'])
        
    def load_encoder(self, save_path):
        self.encoder = keras.models.load_model(save_path)
        
    def load_decoder(self, save_path):
        self.decoder = keras.models.load_model(save_path)
    
    def kl_reconstruction_loss(self, true, pred):
        """
        Loss function for Kullback-Leibler + Reconstruction loss

        Args:
            true: True values
            pred: Predicted values
        Returns:
            Loss = Kullback-Leibler + Reconstruction loss
        """
        mse = MeanSquaredError()
        self.reconstruction_loss = mse(true, pred)
        self.kl_loss = backend.mean(-0.5 * backend.sum( 1.0 + self.sigma - backend.square(self.mu) - backend.exp(self.sigma), axis=-1))
        self.loss = self.reconstruction_loss + self.kl_loss

        return self.loss
    
    def train(self, training_data_in, training_data_out, validation_data_in, validation_data_out,
              epochs:int, batch_size:int, callbacks:list, verbosity:int=0):
        self.vae.fit(training_data_in, training_data_out,
                     validation_data = (validation_data_in, validation_data_out),
                     epochs = epochs, batch_size = batch_size, 
                     callbacks = callbacks, verbose = verbosity)
    
    def evaluate(self, test_data_in, test_data_out, verbosity:int=0):
        loss, mse = self.vae.evaluate(test_data_in, test_data_out, verbose=verbosity)
        return (loss, mse)

In [22]:
class FIA_VAE_tune:
    def __init__(self, X, test_size:float, configuration_space:ConfigurationSpace, model_builder,
                 log_dir:str, batch_size:int=16, verbosity:int=0, gpu:bool=False):
        self.configuration_space = configuration_space
        self.model_builder = model_builder
        self.training_data, self.test_data = train_test_split(X, test_size=test_size)
        self.batch_size = batch_size
        self.log_dir = log_dir
        self.verbosity = verbosity

    def train(self, config: Configuration, seed: int = 0, budget:int=25) -> float:
        """
        Method to train the model

        Args:
            config: Configuration to be trained upon
            seed: initializing seed
            budget: number of epochs to be used in training
        
        Returns:
            Average loss of the model
        """
        keras.utils.set_random_seed(seed)

        # Definition
        model = self.model_builder(config)

        # Fitting
        callbacks = []
        if self.verbosity >= 1:
            log_dir = os.path.join(self.log_dir,  datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
            callbacks.append( TensorBoard(log_dir=log_dir, write_graph=True, write_images=True,
                                          update_freq='epoch') )

        model.train(training_data_in=self.training_data, training_data_out=self.training_data,
                    validation_data_in=self.training_data, validation_data_out=self.training_data,
                    epochs=int(budget), batch_size=self.batch_size,
                    callbacks=callbacks, verbosity=self.verbosity)


        # Evaluation
        loss, mse = model.evaluate(self.test_data, self.test_data, verbosity=self.verbosity)
        
        # Clearing model parameters
        keras.backend.clear_session()
                
        return loss
    
fia_vae_tune = FIA_VAE_tune( X, test_size=0.2, configuration_space=configuration_space, model_builder=FIA_VAE,
                                batch_size=64, log_dir=os.path.join(outdir, "log"), verbosity=verbosity, gpu=False )

In [23]:
scenario = Scenario( fia_vae_tune.configuration_space, deterministic=True,
                     n_trials=20, min_budget=2, max_budget=100,
                     n_workers=1, output_directory=outdir,
                     walltime_limit=np.inf, cputime_limit=np.inf, trial_memory_limit=None )   # Max RAM in Bytes (not MB)
                    
initial_design = MultiFidelityFacade.get_initial_design(scenario, n_configs=10)
intensifier = Hyperband(scenario, incumbent_selection="highest_budget")
facade = MultiFidelityFacade( scenario, fia_vae_tune.train, 
                              initial_design=initial_design, intensifier=intensifier,
                              overwrite=overwrite, logging_level=30-verbosity*10 )
time_step(message=f"SMAC defined. Overwriting: {overwrite}", verbosity=verbosity)

[INFO][abstract_initial_design.py:82] Using `n_configs` and ignoring `n_configs_per_hyperparameter`.
[INFO][abstract_initial_design.py:95] Reducing the number of initial configurations from 10 to 5 (max_ratio == 0.25).
[INFO][abstract_initial_design.py:147] Using 5 initial design configurations and 0 additional configurations.
SMAC defined. Overwriting: False (33.57362484931946s)


### Search

In [24]:
incumbent = run_optimization(facade=facade, smac_model=fia_vae_tune, verbose_steps=10, verbosity=verbosity)

Starting search:


  0%|          | 0/10 [00:00<?, ?it/s]

[INFO][abstract_intensifier.py:305] Using only one seed for deterministic scenario.
[INFO][successive_halving.py:164] Successive Halving uses budget type BUDGETS with eta 3, min budget 2, and max budget 100.
[INFO][successive_halving.py:323] Number of configs in stage:
[INFO][successive_halving.py:325] --- Bracket 0: [27, 9, 3, 1]
[INFO][successive_halving.py:325] --- Bracket 1: [12, 4, 1]
[INFO][successive_halving.py:325] --- Bracket 2: [6, 2]
[INFO][successive_halving.py:325] --- Bracket 3: [4]
[INFO][successive_halving.py:327] Budgets in stage:
[INFO][successive_halving.py:329] --- Bracket 0: [3.7037037037037033, 11.11111111111111, 33.33333333333333, 100.0]
[INFO][successive_halving.py:329] --- Bracket 1: [11.11111111111111, 33.33333333333333, 100.0]
[INFO][successive_halving.py:329] --- Bracket 2: [33.33333333333333, 100.0]
[INFO][successive_halving.py:329] --- Bracket 3: [100.0]
Train on 54 samples, validate on 54 samples


2024-04-10 20:18:58.506335: W tensorflow/c/c_api.cc:305] Operation '{name:'training_6/Nadam/latent_mu_4/bias/m/Assign' id:3944 op device:{requested: '', assigned: ''} def:{{{node training_6/Nadam/latent_mu_4/bias/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training_6/Nadam/latent_mu_4/bias/m, training_6/Nadam/latent_mu_4/bias/m/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Epoch 1/3
54/54 [==============================] - ETA: 0s - loss: 1.3775e-05 - mse: 1.3773e-05

...

54/54 [==============================] - ETA: 0s - loss: 0.0013 - mse: 9.6538e-06

2024-04-10 20:19:11.715709: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2024-04-10 20:19:11.717029: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.


54/54 [==============================] - 1s 28ms/sample - loss: 0.0013 - mse: 9.6538e-06 - val_loss: 5.6824e-05 - val_mse: 7.4119e-06
Epoch 3/3
54/54 [==============================] - ETA: 0s - loss: 5.7198e-05 - mse: 7.5541e-06

 20%|██        | 2/10 [00:15<01:03,  7.88s/it]


KeyboardInterrupt: 